# Part 1

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy
import random

-different value distributions d
-different numbers of bidders in each auction (e.g., do the results change between the case of two bidders in the auction and five or ten bidders in the auction?) 
-different number of items for sale in each round (selling 4 items with the truthful 5th price auction)

## Generation Methods

In [13]:
def generate_random_bidlist(num_bidders, num_rounds):
    bid_list = []
    for i in range(num_rounds):
        bidder_bids = [random.random() for i in range(num_bidders)]
        bid_list.append(bidder_bids)
    return bid_list

def generate_quadratic_bidlist(num_bidders, num_rounds):
    bid_list = []
    for i in range(num_rounds):
        bidder_bids = [math.sqrt(random.random()) for i in range(num_bidders)]
        bid_list.append(bidder_bids)
    return bid_list

def generate_exponential_bids(num_bidders, num_rounds):
    bids_uncapped = numpy.random.exponential(scale=1.0, size=num_bidders)
    for index in range(num_bidders):
        if bids_uncapped[index] > 10:
            bids_uncapped[index] = 10
    return bids_uncapped

def generate_exponential_bidlist(num_bidders, num_rounds):
    return [generate_exponential_bids(num_bidders, num_rounds) for i in range(num_rounds)]

def generate_linear_discretization(val, epsilon):
    action_list = []
    k = val / epsilon
    
    #calculate number of incremented bids
    num_integ = math.ceil(k)
    
    #value of each linear increment 
    increm = k / num_integ
    
    bid = val
    j = 0
    while bid >= 0:
        action_list.append(bid)
        j = j + 1
        bid = val - (epsilon * (increm * j))
    return action_list

# this doesn't work yet, still trying to figure it out
def generate_geometric_discretization(val, epsilon):
    action_list = []
    k = 1/math.e * numpy.log(val)
    bid = val
    j = 0
    while bid >= 0:
        if j == 0: 
            #add both the value bid and the bid of value - 1 according to formula
            action_list.append(bid)
            action_list.append(val - pow((1 + epsilon), j))
        else: 
            if bid not in action_list: 
                action_list.append(bid)
                bid = val - pow((1 + epsilon), j)
            else:
                bid = val - pow((1 + epsilon), j)
        j = j + 1
        
    #add a bid of 0 if not already done so
    if 0 not in action_list:
        action_list.append(0)
        
    return action_list

print(generate_linear_discretization(1, .05))

print(generate_geometric_discretization(1, 0.05))

[10, 9.95, 9.9, 9.85, 9.8, 9.75, 9.7, 9.65, 9.6, 9.55, 9.5, 9.45, 9.4, 9.35, 9.3, 9.25, 9.2, 9.15, 9.1, 9.05, 9.0, 8.95, 8.9, 8.85, 8.8, 8.75, 8.7, 8.65, 8.6, 8.55, 8.5, 8.45, 8.4, 8.35, 8.3, 8.25, 8.2, 8.15, 8.1, 8.05, 8.0, 7.949999999999999, 7.9, 7.85, 7.8, 7.75, 7.699999999999999, 7.65, 7.6, 7.55, 7.5, 7.449999999999999, 7.4, 7.35, 7.3, 7.25, 7.199999999999999, 7.15, 7.1, 7.05, 7.0, 6.949999999999999, 6.9, 6.85, 6.8, 6.75, 6.699999999999999, 6.65, 6.6, 6.55, 6.5, 6.449999999999999, 6.4, 6.35, 6.3, 6.25, 6.199999999999999, 6.15, 6.1, 6.05, 6.0, 5.95, 5.8999999999999995, 5.85, 5.8, 5.75, 5.7, 5.6499999999999995, 5.6, 5.55, 5.5, 5.45, 5.3999999999999995, 5.35, 5.3, 5.25, 5.199999999999999, 5.1499999999999995, 5.1, 5.05, 5.0, 4.949999999999999, 4.8999999999999995, 4.85, 4.8, 4.75, 4.699999999999999, 4.6499999999999995, 4.6, 4.55, 4.5, 4.449999999999999, 4.3999999999999995, 4.35, 4.3, 4.25, 4.199999999999999, 4.1499999999999995, 4.1, 4.05, 4.0, 3.9499999999999993, 3.8999999999999995, 3.8

## Algorithm Classes

In [ ]:
class ExponentialWeights:
    
    def __init__(self, epsilon, num_actions=2):
        self.weights_vector = [1 for i in range(num_actions)]
        self.totals_by_round = []
        self.payoffs_by_round = []
        self.choices_by_round = []
        self.actions_list = [i for i in range(num_actions)]
        self.epsilon = epsilon
        self.num_actions = num_actions
        
    def reset_instance(self, num_actions=2):
        self.weights_vector = [1 for i in range(num_actions)]
        self.totals_by_round = []
        self.payoffs_by_round = []
        self.choices_by_round = []
        self.actions_list = [i for i in range(num_actions)]
        self.num_actions = num_actions

    
    def choose_action(self, max_payoff):
        # find weights
        current_weights = [None for i in range(self.num_actions)]
        for action in range(self.num_actions):
            if self.totals_by_round == []:
                V_last = 0
            else:
                V_last = self.totals_by_round[-1][action]
            exp = V_last / max_payoff
            current_weights[action] = pow(1 + self.epsilon, exp)
        # randomly select from actions using weights as probabilities
        selected_action = random.choices(self.actions_list, weights=current_weights, k=1)[0]
        self.choices_by_round.append(selected_action)
        self.weights_vector.append(current_weights)
        #print('current weights', current_weights)
        return selected_action
    
    def process_payoff(self, selected_payoff, payoff_list):
        # add new payoffs to totals, add payoff choice this round to payoffs matrix
        self.payoffs_by_round.append(selected_payoff)
        if self.totals_by_round == []: 
            self.totals_by_round.append([payoff_list[i] for i in range(self.num_actions)])
        else:
            last_round_totals = self.totals_by_round[-1]
            self.totals_by_round.append([last_round_totals[i] + payoff_list[i] for i in range(self.num_actions)])
                
            
    #NOTE: totals_by_round[-1] at the end of the simulation will help find 'OPT'

In [ ]:
class FTL:
    
    def __init__(self, num_actions=2):
        self.totals_by_round = []
        self.payoffs_by_round = []
        self.choices_by_round = []
        self.actions_list = [i for i in range(num_actions)]
        self.num_actions = num_actions
        
    def reset_instance(self, num_actions=2):
        self.totals_by_round = []
        self.payoffs_by_round = []
        self.choices_by_round = []
        self.actions_list = [i for i in range(num_actions)]
        self.num_actions = num_actions
    
    def choose_action(self, max_payoff):
        # randomly select from actions using highest total payoff so far
        if self.totals_by_round != []:
            selected_action = self.totals_by_round[-1].index(max(self.totals_by_round[-1]))
            self.choices_by_round.append(selected_action)
            return selected_action
        else:
            selected_action = random.randrange(0, self.num_actions)
            return selected_action
    
    def process_payoff(self, selected_payoff, payoff_list):
        # add new payoffs to totals, add payoff choice this round to payoffs matrix
        self.payoffs_by_round.append(selected_payoff)
        if self.totals_by_round == []: 
            self.totals_by_round.append([payoff_list[i] for i in range(self.num_actions)])
        else:
            last_round_totals = self.totals_by_round[-1]
            self.totals_by_round.append([last_round_totals[i] + payoff_list[i] for i in range(self.num_actions)])
                
            
    #NOTE: totals_by_round[-1] at the end of the simulation will help find 'OPT'

## Auction Simulator

In [ ]:
# helpers to find regret of an algorithm
def sum_to_round_i(alg_payoffs, current_round):
    total = 0
    for i in range(current_round):
        total += alg_payoffs[i]
    return total

def individual_regrets(alg_payoffs, round_totals):
    final_payoffs = round_totals[-1]
    opt_action = final_payoffs.index(max(final_payoffs))
    individual_regrets = [0 for i in range(len(alg_payoffs))]
    for round_x in range((len(alg_payoffs))):
        individual_regrets[round_x] = (round_totals[round_x][opt_action] - sum_to_round_i(alg_payoffs, round_x)) / (round_x + 1)
    return individual_regrets

def find_payoff(r_price, bid_list):
    sorted_bids = sorted(bid_list, reverse=True)
    max_bid = sorted_bids[0]
    second_bid = sorted_bids[1]
    if r_price > second_bid and r_price <= max_bid:
        return r_price
    elif r_price > second_bid and r_price > max_bid:
        return 0
    else:
        return second_bid

def auction_simulator(alg, bid_lists, num_rounds, max_bid, price_discretization):
    num_actions = len(price_discretization)
    for bid_list in bid_lists:
        # have the algorithm select a bid
        alg_action = alg.choose_action(max_bid)
        alg_price = price_discretization[alg_action]
        
        # calculate payoff list for each reserve price on the discretization
        payoff_list = []
        for reserve_price in price_discretization:
            payoff_list.append(find_payoff(reserve_price, bid_list))
        alg_payoff = payoff_list[alg_action]
        alg.process_payoff(alg_payoff, payoff_list)
    
    # calculate regrets and payoffs
    alg_regrets = individual_regrets(alg.payoffs_by_round, alg.totals_by_round)
    alg_payoffs = alg.payoffs_by_round
    
    return alg_regrets, alg_payoffs

#bid_lists = [[0, 1], [0, .75], [0, .5], [0, .25]]
#alg = ExponentialWeights(0.0, len(generate_linear_discretization(0, 1, 0.25)))
#alg_regrets, alg_payoffs = auction_simulator(alg, bid_lists, 4, 1, generate_linear_discretization(0, 1, 0.25))
#print(alg_regrets)
#print(alg_payoffs)
#print(alg.weights_vector[-1])
#print(alg.choices_by_round)
#print(alg.totals_by_round)

## Auction Monte Carlo Trials

In [ ]:
## Auction Monte Carlo Trials
def auction_trial(alg, auction_list, num_rounds, max_bid, price_discretization):
    alg_avg_regret_per_round = None
    alg_avg_payoff_per_round = None

    for auction in auction_list:
        # find which trial number we are on
        n = auction_list.index(auction)
                    
        # run matchup and find regret lists
        new_alg_regrets, new_alg_payoffs = auction_simulator(alg, auction, num_rounds, max_bid, price_discretization)
        
        # update average regrets
        if alg_avg_regret_per_round == None:
            alg_avg_regret_per_round = new_alg_regrets
        else:
            for i in range(len(alg_avg_regret_per_round)):
                alg_avg_regret_per_round[i] = ((n * alg_avg_regret_per_round[i]) + new_alg_regrets[i]) / (n + 1) 
        
        # update average payoffs
        if alg_avg_payoff_per_round == None:
            alg_avg_payoff_per_round = new_alg_payoffs
        else:
            for i in range(len(alg_avg_regret_per_round)):
                alg_avg_payoff_per_round[i] = ((n * alg_avg_payoff_per_round[i]) + new_alg_payoffs[i]) / (n + 1) 
        
        # reset alg internally stored values
        alg.reset_instance(num_actions=len(price_discretization))
        #print('final weights', alg.weights_vector)
    return alg_avg_regret_per_round, alg_avg_payoff_per_round

#bid_lists = [[0, 1], [0, .75], [0, .5], [0, .25]]
#alg = ExponentialWeights(1.0, len(generate_linear_discretization(0, 1, 0.25)))
#alg_regrets, alg_payoffs = auction_trial(alg, [bid_lists, bid_lists], 4, 1, generate_linear_discretization(0, 1, 0.25))
#print(alg_regrets)
#print(alg_payoffs)

## Visualization of Regrets

In [ ]:
def visualize_rounds(alg_regrets, rounds, lr, plot_title, alg_name, trial_type, y_label):
    
    file_name = trial_type + '_' + alg_name + "_" + f'{lr}' + '.png'
    
    x = numpy.array(list(range(0, rounds)))
    y_1 = numpy.array(alg_regrets)
    plt.plot(x, y_1, label='{alg_name}, learning rate = {lr}'.format(alg_name=alg_name, lr = lr), linewidth=1)
    plt.xlabel("Round")
    plt.ylabel(y_label)
    plt.title(plot_title)
    plt.legend(loc='best', prop={'size': 7})
    
    plt.savefig(file_name)

    plt.show()


# Trials on Different Bidder Generation Methods, Discretizations, Number of Bidders

In [ ]:
# PARAMETERS
NUM_TRIALS = 1000
NUM_ROUNDS = 500

# 2 bidder random values auction trial Epsilon = 1
num_bidders = 2
auction_list = []
min_payoff, max_payoff = 0, 1
# perform 500 trials each with 500 rounds and 2 bidders
for i in range(NUM_TRIALS):
    auction_list.append(generate_random_bidlist(num_bidders, NUM_ROUNDS))
price_discretization = generate_linear_discretization(0, 1, 0.01)
alg = ExponentialWeights(1.0, len(price_discretization))
alg_regrets, alg_payoffs = auction_trial(alg, auction_list, NUM_ROUNDS, max_payoff, price_discretization)
print(alg_regrets[-1])
print(alg_payoffs[-1])
print('avg 1', sum(alg_payoffs) / len(alg_payoffs))
visualize_rounds(alg_regrets, NUM_ROUNDS, .5, "demo", "EW", "Randomized Auction", "Regret per Round")
visualize_rounds(alg_payoffs, NUM_ROUNDS, .5, "demo", "EW", "Randomized Auction", "Payoff per Round")


# 2 bidder random values auction trial Epsilon = 0
num_bidders = 2
auction_list = []
min_payoff, max_payoff = 0, 1
# perform 500 trials each with 500 rounds and 2 bidders
for i in range(NUM_TRIALS):
    auction_list.append(generate_random_bidlist(num_bidders, NUM_ROUNDS))
price_discretization = generate_linear_discretization(0, 1, 0.01)
alg = ExponentialWeights(0, len(price_discretization))
alg_regrets, alg_payoffs = auction_trial(alg, auction_list, NUM_ROUNDS, max_payoff, price_discretization)
print(alg_regrets[-1])
print(alg_payoffs[-1])
print('avg 2', sum(alg_payoffs) / len(alg_payoffs))
visualize_rounds(alg_regrets, NUM_ROUNDS, 0, "demo", "EW", "Randomized Auction", "Regret per Round")
visualize_rounds(alg_payoffs, NUM_ROUNDS, 0, "demo", "EW", "Randomized Auction", "Payoff per Round")


# Part 2

### Generating Meet Values